<a href="https://colab.research.google.com/github/TheRadDani/AI/blob/main/11_RLHF_Training_for_CustomDataset_for_AnyModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00


In [2]:
## Import libraries
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from trl import RewardTrainer, SFTTrainer, RewardConfig
from datasets import Dataset
import json
import pandas as pd
from transformers import Trainer, TrainingArguments

In [3]:
!wget https://huggingface.co/datasets/CarperAI/openai_summarize_comparisons/resolve/main/data/test-00000-of-00001-0845e2eec675b16a.parquet
# after that rename to test.parquet

--2024-03-01 04:11:39--  https://huggingface.co/datasets/CarperAI/openai_summarize_comparisons/resolve/main/data/test-00000-of-00001-0845e2eec675b16a.parquet
Resolving huggingface.co (huggingface.co)... 65.8.243.46, 65.8.243.90, 65.8.243.92, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.46|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/02/84/02842250b699bbde4e980e9206f5045df542f9ca374595cc86b8d57189641ff5/a8c5d73113a2c7119faacd44e43959f74e5b59e181f3da57caa918c907ac6620?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27test-00000-of-00001-0845e2eec675b16a.parquet%3B+filename%3D%22test-00000-of-00001-0845e2eec675b16a.parquet%22%3B&Expires=1709525499&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTUyNTQ5OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8wMi84NC8wMjg0MjI1MGI2OTliYmRlNGU5ODBlOTIwNmY1MDQ1ZGY1NDJmOWNhMzc0NTk1Y2M4NmI

In [4]:
%%bash
mv test-00000-of-00001-0845e2eec675b16a.parquet test.parquet

### Dataset for Training : https://huggingface.co/datasets/CarperAI/openai_summarize_tldr

### Dataset for Testing : https://huggingface.co/datasets/CarperAI/openai_summarize_comparisons

In [5]:
Model_name = "bigcode/tiny_starcoder_py"
Data_path = "test.parquet"

In [6]:
# Read the file
df  = pd.read_parquet(Data_path)
df = df[:10]
raw_dataset = Dataset.from_pandas(df)
raw_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 10
})

### Tokenize the dataset to feed in the model

In [9]:
tokenizer = AutoTokenizer.from_pretrained(Model_name)
model = AutoModelForCausalLM.from_pretrained(Model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/657M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### Tokenize the dataset to feed in the model

In [10]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def formatting_func(examples):
    kwargs = {
        "padding": "max_length",
        "truncation": True,  # Ensure truncation is set to True
        "max_length": 256,
        "return_tensors": "pt"
    }

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = examples["prompt"] + "\n" + examples["chosen"]
    prompt_plus_rejected_response = examples["prompt"] + "\n" + examples["rejected"]

    # Then tokenize these modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0],
        "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0],
        "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }


### Map the function to data

In [11]:
formatted_dataset = raw_dataset.map(formatting_func)
formatted_dataset = formatted_dataset.train_test_split()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [174]:
model.config

GPTBigCodeConfig {
  "_name_or_path": "bigcode/tiny_starcoder_py",
  "activation_function": "gelu_pytorch_tanh",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": 3072,
  "n_layer": 20,
  "n_positions": 8192,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "resid_pdrop": 0.1,
  "scale_attention_softmax_in_fp32": true,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.38.1",
  "use_cache": true,
  "validate_runner_input": tr

In [12]:
### Loading the TRL reward trainer and training the trainer
training_args = RewardConfig(
        output_dir="tinystarcoder-rlhf-model",
        num_train_epochs=1,
        logging_steps=10,
        gradient_accumulation_steps=1,
        save_strategy="steps",
        evaluation_strategy="steps",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=1,
        eval_accumulation_steps=1,
        eval_steps=500,
        save_steps=500,
        max_length = 512,
        warmup_steps=100,
        logging_dir="./logs",
        learning_rate=1e-5,
        save_total_limit=1,
        use_cpu=True
    )

In [13]:
trainer = RewardTrainer(model=model,
                        tokenizer=tokenizer,
                        train_dataset=formatted_dataset['train'],
                        eval_dataset=formatted_dataset['test'],
                        args= training_args
                        )
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:189: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


TrainOutput(global_step=4, training_loss=0.7626696228981018, metrics={'train_runtime': 84.7869, 'train_samples_per_second': 0.083, 'train_steps_per_second': 0.047, 'total_flos': 0.0, 'train_loss': 0.7626696228981018, 'epoch': 1.0})

In [50]:
trainer.evaluate()

IndexError: index 1 is out of bounds for axis 1 with size 1

In [51]:
from huggingface_hub import login
login()

In [154]:
trainer.push_to_hub("RLHF model of StarCoder")

model.safetensors:   0%|          | 0.00/657M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/TheRadDani/tinystarcoder-rlhf-model/commit/ab802625059daf9edc5464c3222722026c5da220', commit_message='RLHF model of StarCoder', commit_description='', oid='ab802625059daf9edc5464c3222722026c5da220', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
trainer.save_model("tinystarcoder-rlhf-model")

### Policy Model

In [156]:
help(trainer.push_to_hub)

Help on method push_to_hub in module transformers.trainer:

push_to_hub(commit_message: Optional[str] = 'End of training', blocking: bool = True, **kwargs) -> str method of trl.trainer.reward_trainer.RewardTrainer instance
    Upload `self.model` and `self.tokenizer` to the 🤗 model hub on the repo `self.args.hub_model_id`.
    
    Parameters:
        commit_message (`str`, *optional*, defaults to `"End of training"`):
            Message to commit while pushing.
        blocking (`bool`, *optional*, defaults to `True`):
            Whether the function should return only when the `git push` has finished.
        kwargs (`Dict[str, Any]`, *optional*):
            Additional keyword arguments passed along to [`~Trainer.create_model_card`].
    
    Returns:
        The URL of the repository where the model was pushed if `blocking=False`, or a `Future` object tracking the
        progress of the commit if `blocking=True`.



In [14]:
hugging_face_model = "TheRadDani/tinystarcoder-rlhf-model"

In [15]:
## inference the model
rm_model = AutoModelForCausalLM.from_pretrained(hugging_face_model, device_map="auto",)
tokenizer = AutoTokenizer.from_pretrained(hugging_face_model)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/657M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

In [16]:
def get_score(model, tokenizer, prompt, response):

    instructions = tokenizer.encode_plus(prompt,
                                       response,
                                       padding="max_length",
                                       max_length=256,
                                       return_tensors="pt",
                                        truncation=True)
    with torch.no_grad():
        outputs = model(**instructions)

    logits = outputs[0]

    return logits

In [17]:
prompt_str = df.iloc[0]["prompt"]
example_prefered_response = df.iloc[0]["chosen"]
example_unprefered_response = df.iloc[0]["rejected"]

In [18]:
loss1 = get_score(model, tokenizer, prompt_str, example_prefered_response)
loss2= get_score(model, tokenizer, prompt_str, example_unprefered_response)

In [19]:
from torch import nn
loss = -nn.functional.logsigmoid(loss1 - loss2).mean()

In [20]:
tokenizer.decode(torch.max(loss1, axis=-1).indices[0])

'_MEIT_\n           \'2/\n: r RelationshipMy]]0]\nlsriend\n29/M]\n [\n [\n the was to the [\n a friends to\n\n: < [lfriend [ me don a aressed. my10 minutes.\n\n"""1 updated:** girlfriend and through my Facebook.** I the my gir.**. my few of lf**\n\n** went dd for for my gir girirl** I was to findoolpp my my my future** and I was that in\n\n** have ali the of ago she.liting me me girirl. and she had the to me. in me few of years.1viously). was\'t find it was not\n\n** was it for my girl and the was the Facebook. the   she wasand historyirl) was was April,\n to, f that were foughtted. I my more.. f.ing on her.\nirirl isLY I1 girirllfriend is 19 months. to my Facebook and. my permission. I her messages. her.lirty with her coupleirl.\n found up with me after she through more with\n'

### Policy Model

In [21]:
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from trl import RewardTrainer, SFTTrainer
from datasets import Dataset
import json
import pandas as pd
from transformers import Trainer, TrainingArguments
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model

In [22]:
##model path
MODEL_PATH = "bigcode/tiny_starcoder_py"
DATA_PATH = "test.parquet"

In [23]:
df = pd.read_parquet(DATA_PATH)
df = df[:1000]
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1000
})

In [24]:
from transformers import AutoTokenizer, pipeline

In [28]:
rf_model_path = "tinystarcoder-rlhf-model"
starcoder_model = AutoModelForCausalLMWithValueHead.from_pretrained(rf_model_path)
starcoder_model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(rf_model_path)
starcoder_tokenizer = AutoTokenizer.from_pretrained(rf_model_path)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

In [29]:
txt_in_len = 5
txt_out_len = 32
seed = 1

dataset = dataset.map(
    lambda x: {"input_ids": tokenizer.encode(" " + x["chosen"], return_tensors="pt", truncation=True, padding="max_length", max_length=32)[0]},
    batched=False,
)
dataset = dataset.map(lambda x: {"query": tokenizer.decode(x["input_ids"])}, batched=False)
dataset = dataset[:20480]
from datasets import Dataset

dataset = Dataset.from_dict(dataset)
dataset.set_format("pytorch")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [27]:
dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected', 'input_ids', 'query'],
    num_rows: 1000
})

In [30]:
sentiment_pipe_kwargs = {"top_k": None, "function_to_apply": "none"} #logits

config = PPOConfig(
    model_name=MODEL_PATH, steps=51200, learning_rate=1.41e-5, remove_unused_columns=True
)

txt_in_len = 5
txt_out_len = 20
seed = 1

In [31]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [32]:
import torch
optimizer = torch.optim.SGD(starcoder_model.parameters(), lr=config.learning_rate)
ppo_trainer = PPOTrainer(config, starcoder_model, starcoder_model, starcoder_tokenizer, dataset=dataset, data_collator=collator, optimizer=optimizer)

In [87]:
#for i in ppo_trainer.dataloader:
#  print(i)

In [33]:
ctrl_str = ["[negative]", "[positive]"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # this should be handled by accelerate
ctrl_tokens = dict((s, starcoder_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)) for s in ctrl_str)

In [34]:
def pos_logit_to_reward(logit, task):
    """
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [neutral]: reward = -2*abs(logit)+4
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i] == "[negative]":
            logit[i] = -logit[i]
        elif task[i] == "[positive]":
            pass
        else:
            raise ValueError("task has to be in [0, 1, 2]!")
    return logit

In [35]:
pos_logit_to_reward(torch.Tensor([4, 4]), ctrl_str)

tensor([-4.,  4.])

In [36]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": starcoder_tokenizer.eos_token_id,
    "max_new_tokens": 32,
    "eos_token_id": -1,
}

In [37]:
def get_score(model, tokenizer, responses):
    positive_logist = []
    for i in responses:
        instructions = tokenizer.encode_plus(
                                           i,
                                           padding="max_length",
                                           max_length=32,
                                           return_tensors="pt")
        with torch.no_grad():
            outputs = model(**instructions)

        logits = outputs[0].mean()
        positive_logist.append(logits)

    return positive_logist


In [38]:
responses =["heelow how are you", "__IT_\nr/\n: r RelationshipRelationship]]0]\nlsriend\n2//M]\n [ [ a\n the was to the [. a friends to\n\n:\n [lfriend [ me have a aried in his19 minutes.\n\nWhat Modified:** girlfriend was through the Facebook.. I my my friends.**** my  of lf**\n\n** was d1ing for my few personirl** I had for findoolpping my my the future** but I was that in\n\n** have ali  of to she  tolirt my me girl. and she found my about my.. me few of gir.1viously). was\'t find her was).\n\n** was it about my twoirl and the had  Facebook. the  and she gand historyirl) was in April,\n to, find, were flirted. I a messages.. f.ing on her.\n girlM\n; I1 girirllfriend and the19 months. to my Facebook.. my permission. she her messages. my.lirty with my fewirl.\n found her with me. I through more with\n"]
get_score(starcoder_model, tokenizer, responses)

[tensor(-4.3099), tensor(-5.8219)]

In [ ]:
from random import choices
from tqdm import tqdm
import time
import numpy as np

for epoch in range(1):
  for batch in tqdm(ppo_trainer.dataloader):


    (logs, game_data,) = (
             dict(),
             dict(),
    )
    print()
    print(ctrl_str)
    #### prepend a random control token
    task_list = choices(ctrl_str, k=config.batch_size)
    game_data["query"] = [t + q for t, q in zip(task_list, batch["query"])]

    query_tensors = [torch.cat((ctrl_tokens[t], input_ids)) for t, input_ids in zip(task_list, batch["input_ids"])]

    #### get response from gpt2
    response_tensors = []
    for query in query_tensors:
      response = ppo_trainer.generate(query, **generation_kwargs)
      response_tensors.append(response.squeeze()[-txt_out_len:])
      print(response_tensors)
      game_data["response"] = [starcoder_tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### sentiment analysis
    texts = [q + r for q, r in zip(batch["query"], game_data["response"])]
    logits = get_score(starcoder_model,starcoder_tokenizer, texts)
    rewards = pos_logit_to_reward(logits, task_list)

    #### Run PPO training
    t = time.time()
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

    for cs in ctrl_str:
      key = "env/reward_" + cs.strip("[]")
      stats[key] = np.mean([r.cpu().numpy() for r, t in zip(rewards, task_list) if t == cs])
    ppo_trainer.log_stats(stats, game_data, rewards)


  0%|          | 0/7 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
           81,  4189,   346,   324,  1459,   440,  5261,    81,   777,  9999]), tensor([  322,  7592,   996, 13710,   597, 12791,   203,     0,     1,     3,
           30,   225,    39,    30,   225,    39,    30, 14504,   322,  5168]), tensor([37379,  9584, 21672,  1039, 26736, 10809, 35589, 11894,    31,  3275,
         6479,  6963, 26522, 10809,    30,  9295,  8949,  6479,   392,   291]), tensor([  584,  9105,    81,  5137,    81, 12063,   280,  8594,    26, 20251,
           30,  1202,    47,  5962,   298,    44,   298,  2867,  4548,   614]), tensor([ 1395,   526,   312,  1742,   432,   322, 38212,    32,   203,  9592,
          203,     0,     5,  1614,   927,    33,  1614,   927,    33,  5410]), tensor([  745,  2992,  5273, 11089,   544,  7791,   291,   345,    81,   297,
           81,   469,   279,   280,   443,  1038,   717,    32,  2872,    26]), tensor([ 1932, 30486,   556,   345,   305,   493, 39269,   645, 10320,    32,
 

In [41]:
###saving the model
starcoder_model.save_pretrained("StarCoderTinyrhlfmodel/")
starcoder_tokenizer.save_pretrained("StarCoderTinyrhlfmodel/")

('StarCoderTinyrhlfmodel/tokenizer_config.json',
 'StarCoderTinyrhlfmodel/special_tokens_map.json',
 'StarCoderTinyrhlfmodel/vocab.json',
 'StarCoderTinyrhlfmodel/merges.txt',
 'StarCoderTinyrhlfmodel/added_tokens.json',
 'StarCoderTinyrhlfmodel/tokenizer.json')

In [42]:
from transformers import pipeline, set_seed
model_path = "StarCoderTinyrhlfmodel/"
set_seed(42)
pipe = pipeline("text-generation",model=model_path, tokenizer=model_path, max_length=30, num_return_sequences=5)

Some weights of the model checkpoint at StarCoderTinyrhlfmodel/ were not used when initializing GPTBigCodeForCausalLM: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPTBigCodeForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPTBigCodeForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
